# Création d'une session Spark

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
            .appName('Prediction_Attrition_Client')
            .master('local[*]')
            .getOrCreate())

# Vérification de la version de Spark

In [3]:
print(spark.version)

4.0.1


# Chargement des données brutes 

In [9]:
df = spark.read.csv("dataset.csv", header = True, inferSchema = True)